In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
features = train.copy()
target = train.pop("Survived")

In [3]:
sex_map = {
    "male": 0,
    "female": 1
}
features = features.replace(sex_map)
test = test.replace(sex_map)
# Replace Missing age with average age instead maybe
age = features["Age"] + test["Age"]
age_mean = age.mean()
features['Age'] = features['Age'].fillna(age_mean)
test['Age'] = test['Age'].fillna(age_mean)
test = test.fillna(0)
# Look for names that contain Master or other titles and create a new feature for that
id = test['PassengerId']
train_names = features["Name"]
test_names = test["Name"]
train_title = []
test_title = []
for name in train_names:
    if "Mr" in name:
        train_title.append(1)
    elif "Miss" in name:
        train_title.append(2)
    elif "Mrs" in name:
        train_title.append(3)
    elif "Master" in name:
        train_title.append(4)
    else:
        train_title.append(0)
for name in test_names:
    if "Mr" in name:
        test_title.append(1)
    elif "Miss" in name:
        test_title.append(2)
    elif "Mrs" in name:
        test_title.append(3)
    elif "Master" in name:
        test_title.append(4)
    else:
        test_title.append(0)    

features = features.drop(columns=["Name", "Survived", "Ticket", "PassengerId", "Cabin"])
test = test.drop(columns=["Name", "Ticket", "PassengerId", "Cabin"])
features["Embarked"] = pd.Categorical(features["Embarked"])
test["Embarked"] = pd.Categorical(test["Embarked"])
features["Embarked"] = features.Embarked.cat.codes
test["Embarked"] = test.Embarked.cat.codes

In [5]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(features, target)
results = rf.predict(test)

In [6]:
results
write = {
    "PassengerId" : id,
    "Survived" : results
}
write_df = pd.DataFrame(data=write)
write_df.to_csv("results.csv", index=False)

In [7]:
write_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
